In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.common.keys import Keys
import numpy as np
import time
from datetime import datetime
from bs4 import BeautifulSoup
import re

In [2]:
path = r'C:/Users/Tinatin/.wdm/drivers/chromedriver/win32/83.0.4103.39/chromedriver.exe'

In [3]:
urls = ['https://www.psychologytoday.com/us/therapists/christian/ny/new-york/732713?sid=5ef4185b8066d&ref=1&tr=ResultsPhoto','https://www.psychologytoday.com/us/therapists/african-american/ny/new-york/345502?sid=5eee3e019ee48&ref=9&tr=ResultsPhoto', 'https://www.psychologytoday.com/us/therapists/ny/new-york/411234?sid=5eebcea19c07b&ref=1&tr=ResultsPhoto', 'https://www.psychologytoday.com/us/therapists/ny/new-york/103681?sid=5eebcea19c07b&ref=4&tr=ResultsPhoto', 'https://www.psychologytoday.com/us/therapists/ny/new-york/470297?sid=5eebcea19c07b&ref=3&tr=ResultsName']

In [4]:
final_df = pd.DataFrame(columns=['statement', 'name', 'verified', 'degree', 'addr', 'city', 'state', 'zip', 'phone', 'cost_per_ses','sliding_scale','pay_by','accepted_insurance', 'specs', 'issues', 'mental_health', 'sexuality', 'client_focus_age', 'client_focus_communities', 'treatment_approach_types_of_ther', 'treatment_approach_modality', 'cert', 'cert_date', 'ethnicity', 'language', 'faith'])

### to-do: scrape information about whether they offer video sessions

In [5]:
def scraper(urls, path):
    global final_df
    #iterate over urls
    for url in urls:
        row_results = {}
        driver = Chrome(executable_path= path)
        try:
            driver.get(url)         #open the url in selenium
        except:
            print ('bad url!')
        soup = BeautifulSoup(driver.page_source,'html.parser') 
        
        #wait for the webpage to load
        time.sleep(10)
        #row_results['full_address'] = ', '.join([item for item in (item.strip() for item in soup.find(class_ = "address-data").text.split('\n') if item) if item])
        try:   
            row_results['state'] = soup.find(itemprop = "addressRegion").text
        except:
            print('state field missing')
            row_results['state'] = 'missing'
        try:
            row_results['addr'] = [item for item in (item.strip() for item in soup.find(itemprop = 'streetAddress').text.split('\n') if item) if item][0]
        except:
            print('street address field missing')
            row_results['addr'] = 'missing'
        try:
            row_results['city'] = soup.find(itemprop = 'addressLocality').text
        except:
            row_results['city'] = 'missing'
            print('city field missing')
        try:
            row_results['zip'] = soup.find(itemprop = 'postalcode').text
        except:
            print('zip field missing')
            riw_results['zip'] = 'missing'
        #row_results['header_info'] = ', '.join([item.strip(',') for item in (item.strip() for item in soup.find(id = 'profHdr').text.split('\n') if item) if item])
        row_results['name'] = [item.strip(',') for item in (item.strip() for item in soup.find(id = 'profHdr').text.split('\n') if item) if item][0]
        row_results['degree'] = ', '.join([item.strip(',') for item in (item.strip() for item in soup.find(id = 'profHdr').text.split('\n') if item) if item][1:-1])
        row_results['verified'] = [item.strip(',') for item in (item.strip() for item in soup.find(id = 'profHdr').text.split('\n') if item) if item][-1]
        row_results['phone'] = soup.find(class_ = "phone-number").text
        row_results['statement'] = ''.join([item for item in soup.find(class_ = 'section profile-personalstatement').text.split('\n') if item])
        finances = [item for item in (item.strip() for item in soup.find(id = 'tabs-finances-office').text.split('\n') if item) if item]
        finances.remove(finances[-1])
        #print(finances)
        
        if any("Cost per Session" in s for s in finances):
            for i in range(len(finances)):
                if 'Cost per Session: ' in finances[i]:
                    row_results['cost_per_ses'] = finances[i].replace('Cost per Session: ', '')
        else:
            print('Cost per Session field missing')
            row_results['cost_per_ses'] = 'missing cost_per_ses'
            
        if any('Sliding Scale' in s for s in finances):
            for i in range(len(finances)):
                if 'Sliding Scale: ' in finances[i]:
                    row_results['sliding_scale'] = finances[i].replace('Sliding Scale: ', '')
        else:
            print('Sliding Scale field missing')
            row_results['sliding_scale'] = 'missing sliding_scale'
            
        for i in range(len(finances)):
            if 'Pay By' in finances[i]:
                try:
                    row_results['pay_by'] = finances[i+1]
                except: 
                    print('missing pay_by')
                    row_results['pay_by'] = 'missing pay by'
                    
        if 'Accepted Insurance Plans' not in finances:
                print('insurance field missing')
                row_results['accepted_insurance'] = 'missing insurance'
        else:
            for i in range(len(finances)):
                if 'Accepted Insurance Plans' in finances[i]:
                    row_results['accepted_insurance'] = [i for i in finances[i+1:]]
        #row_results['quals'] = soup.find(class_ = 'profile-qualifications details-section top-border').text
        check = ['Years in Practice:','School:',  'Year Graduated:', 'License and State:']
        quals = soup.find(class_ = 'profile-qualifications details-section top-border').text
        quals_list = [item for item in (item.strip() for item in quals.split('\n') if item) if item][1:]
        if len(quals_list) > 8:
            print('check this link')
        for i in check:
            if i in quals_list:
                row_results[i] = quals_list[quals_list.index(i)+1]
            else:
                row_results[i] = 'missing'
                print(i + ' field missing')
        row_results['specs'] = ', '.join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-top').text.split('\n') if item) if item])
        row_results['issues'] = ', '.join([item for item in (item.strip() for item in soup.find(class_= 'spec-list attributes-issues').text.split('\n') if item) if item][1:])
        try:
            row_results['mental_health'] = ', '.join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-mental-health').text.split('\n') if item) if item][1:])
            
        except:
            print('mental health field missing')
            row_results['mental_health'] = 'missing'
        try:
            row_results['sexuality'] = ', '.join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-sexuality').text.split('\n') if item) if item][1:])
        except:
            print('sexuality field missing')
            row_results['sexuality'] = 'missing'
        try:  
            row_results['client_focus_age'] = ", ".join([item for item in (item.strip() for item in soup.find(class_= 'spec-list attributes-age-focus').text.split('\n') if item) if item][1:])
        except:
            print('client focus age field missing')
            row_results['client_focus_age'] = 'missing'
        try:
            row_results['client_focus_communities'] = ", ".join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-categories').text.split('\n') if item) if item][1:])
        except:
            print('client focus communities field missing')
            row_results['client_focus_communities'] = 'missing'
        try:
            row_results['treatment_approach_types_of_ther'] = ", ".join([item for item in (item.strip() for item in soup.find(class_= 'spec-list attributes-treatment-orientation').text.split('\n') if item) if item][1:])
        except:
            print('treatment approach types of therapy field missing')
            row_results['treatment_approach_types_of_ther'] = 'missing'
        try:
            row_results['treatment_approach_modality'] = ", ".join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-modality').text.split('\n') if item) if item][1:])
        except:
            print('treatment approach modality field missing')
            row_results['treatment_approach_modality'] = 'missing'
        try:
            row_results['cert'] = re.sub('Certificate: ', "", [item for item in (item.strip() for item in soup.find(class_ = 'profile-additional-credentials details-section top-border').text.split('\n') if item) if item][1])

        except:
            print('additional credentials certificate field missing')
            row_results['cert'] = 'missing'
        try:
            row_results['cert_date'] = re.sub('Certificate Date: ', "", [item for item in (item.strip() for item in soup.find(class_ = 'profile-additional-credentials details-section top-border').text.split('\n') if item) if item][2])
        except:
            print('additional credentials certificate date field missing')
            row_results['cert_date'] = 'missing'
        try:   
            row_results['ethnicity'] = ", ".join([item for item in (item.strip() for item in soup.find(class_ = 'spec-subcat attributes-ethnicity-focus').text.split('\n') if item) if item][1:])
        except:
            print('ethnicity field missing')
            row_results['ethnicity'] = 'missing'
        try:
            row_results['language'] = ", ".join([item for item in (item.strip() for item in soup.find(class_ = 'spec-subcat attributes-language').text.split('\n') if item) if item][1:])
        except:
            print('language field is missing')
            row_results['language'] = 'missing'
        try:
            row_results['faith'] = ", ".join([item for item in (item.strip() for item in soup.find(class_ = 'spec-subcat attributes-religion').text.split('\n') if item) if item][1:])
        except:
            print('faith field is missing')
            row_results['faith'] = 'missing'
            
        driver.quit()
        final_df = final_df.append(row_results, ignore_index=True)
        print('---'*10)
        
        

In [ ]:
scraper(urls, path)

Sliding Scale field missing
Years in Practice: field missing
Year Graduated: field missing
mental health field missing
sexuality field missing
client focus communities field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
------------------------------
faith field is missing
------------------------------
street address field missing
Cost per Session field missing
missing pay_by
insurance field missing
Years in Practice: field missing
Year Graduated: field missing
mental health field missing
sexuality field missing
client focus communities field missing
treatment approach modality field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missing


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
final_df